#############################################
# PROJE: Hybrid Recommender System


ID'si verilen kullanıcı için item-based ve user-based recomennder yöntemlerini kullanarak tahmin yapalım.

5 öneri user-based modelden 5 öneri de item-based modelden ele alınız ve nihai olarak 10 öneriyi 2 modelden yapalım.

#############################################

In [46]:
# To do list;

# Görev 1: Verinin Hazırlanması

# Görev 2: Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi

# Görev 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek

# Görev 4: Öneri Yapılacak Kullanıcı ile En Benzer Kullanıcıların Belirlenmesi

# Görev 5: Weighted Average Recommendation Score'un Hesaplanması ve İlk 5 Filmin Tutulması

In [47]:
# Kütüphanemizi import edelim;
import pandas as pd

# Pandas ayarlamalarını yapalım;
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.width', 300)

### Görev 1:

In [48]:
# Movie ve Rating veri setlerini okutalım,

# movieId, film adı ve filmin tür bilgilerini içeren veri seti;
movie = pd.read_csv('datasets/movie.csv')

# Verimizie hızlı bir göz atalım;
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [49]:
# movie df'inin shapesine bakalım;
movie.shape

(27278, 3)

In [50]:
# UserID, film adı, filme verilen oy ve zaman bilgisini içeren veri seti
rating = pd.read_csv('datasets/rating.csv')

# Verimizie hızlı bir göz atalım;
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [51]:
# rating df'inin shapesine bakalım;
rating.shape

(20000263, 4)

In [52]:
# kaç farklı user olduğuna bakalım;
rating["userId"].nunique()

138493

In [53]:
# Rating veri setine filmlerin isimlerini ve türünü movie film setini kullanarak ekleyelim;
df = movie.merge(rating, how="left", on="movieId")

# Yeni df'e hızlı bir bakış atalım;
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [54]:
# Yeni df'in shapesine bakalım;
df.shape

(20000797, 6)

In [55]:
# Her bir film için toplam kaç kişinin oy kullandığını hesaplayalım;
comment_counts = pd.DataFrame(df["title"].value_counts())
comment_counts

,title
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715
...,...
Rapture (Arrebato) (1980),1
"Education of Mohammad Hussein, The (2013)",1
Satanas (2007),1
Psychosis (2010),1


In [56]:
# Bazı filmler için çok az oy kullanılmış, az oy kullanılan filmleri için bir öneride bulunmak zor olacak.
# Bu sebeple bir eşik değer belirleyip bu eşik değerden az sayıda oy alan filmleri df'den çıkaracağız.
# Burada eşik değerimizi 1000 olarak belirliyoruz.

# Toplam oy kullanılma sayısı 1000'in altında olan filmlerin isimlerini rare_movies de tutalım,
# ve veri setinden çıkartalım;

rare_movies = comment_counts[comment_counts["title"] <= 1000].index
common_movies = df[~df["title"].isin(rare_movies)]
common_movies.shape

(17766015, 6)

In [57]:
# common_movies'e hızlı bir bakış atalım;
common_movies.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [58]:
# 1000'den fazla yorum alan kaç film kalmış, bakalım;
common_movies.title.value_counts()

Pulp Fiction (1994)                  67310
Forrest Gump (1994)                  66172
Shawshank Redemption, The (1994)     63366
Silence of the Lambs, The (1991)     63299
Jurassic Park (1993)                 59715
                                     ...  
Scanners (1981)                       1003
Pet Sematary II (1992)                1003
Return to Paradise (1998)             1003
Lincoln Lawyer, The (2011)            1001
Wristcutters: A Love Story (2006)     1001
Name: title, Length: 3159, dtype: int64

In [59]:
# index'te userID'lerin sutunlarda film isimlerinin ve değer olarakta ratinglerin bulunduğu bir pivot table oluşturalım;
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

# Yeni df'ye hızlı bir bakış atalım;
user_movie_df.head()

title   'burbs, The (1989)  (500) Days of Summer (2009)  *batteries not included (1987)  ...And Justice for All (1979)  10 Things I Hate About You (1999)  10,000 BC (2008)  101 Dalmatians (1996)  101 Dalmatians (One Hundred and One Dalmatians) (1961)  102 Dalmatians (2000)  12 Angry Men (1957)  \
userId                                                                                                                                                                                                                                                                                                   
1.0                    NaN                          NaN                             NaN                            NaN                                NaN               NaN                    NaN                                                NaN                         NaN                  NaN   
2.0                    NaN                          NaN                             NaN                            NaN                                NaN               NaN                    NaN                                                NaN                         NaN                  NaN   
3.0                    NaN                          NaN                             NaN                            NaN                                NaN               NaN                    NaN                                                NaN                         NaN                  NaN   
4.0                    NaN                          NaN                             NaN                            NaN                                NaN               NaN                    NaN                                                NaN                         NaN                  NaN   
5.0                    NaN                          NaN                             NaN                            NaN                                NaN               NaN                    NaN                                                NaN                         NaN                  NaN   

title   12 Years a Slave (2013)  127 Hours (2010)  13 Going on 30 (2004)  13th Warrior, The (1999)  1408 (2007)  15 Minutes (2001)  16 Blocks (2006)  17 Again (2009)  1984 (Nineteen Eighty-Four) (1984)  2 Days in the Valley (1996)  2 Fast 2 Furious (Fast and the Furious 2, The) (2003)  \
userId                                                                                                                                                                                                                                                                                          
1.0                         NaN               NaN                    NaN                       NaN          NaN                NaN               NaN              NaN                                 NaN                          NaN                                                NaN       
2.0                         NaN               NaN                    NaN                       NaN          NaN                NaN               NaN              NaN                                 NaN                          NaN                                                NaN       
3.0                         NaN               NaN                    NaN                       NaN          NaN                NaN               NaN              NaN                                 5.0                          NaN                                                NaN       
4.0                         NaN               NaN                    NaN                       NaN          NaN                NaN               NaN              NaN                                 NaN                          NaN                                                NaN       
5.0                         NaN               NaN                    NaN                       NaN          NaN                NaN               NaN              NaN              

In [62]:
# Yukarıdaki işlemleri fonksiyonlaştırmak istersek;

m_path = "datasets/movie.csv"
r_path = "datasets/rating.csv"

def create_user_movie_df(m_path, r_path):
    movie = pd.read_csv(m_path)
    rating = pd.read_csv(r_path)
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

In [63]:
# fonksiyonumuzu deneyelim;
user_movie_df = create_user_movie_df(m_path, r_path)

### Görev 2:

In [64]:
# Rastgele bir kullanıcı id'si tanımlayalım;
random_user = 108170

# Seçilen kullanıcıya ait gözlem birimlerinden oluşan random_user_df adında yeni bir dataframe oluşturunuz.
random_user_df = user_movie_df[user_movie_df.index == random_user]

# Veriye hızlı bir bakış atalım;
random_user_df.head()

title     'burbs, The (1989)  (500) Days of Summer (2009)  *batteries not included (1987)  ...And Justice for All (1979)  10 Things I Hate About You (1999)  10,000 BC (2008)  101 Dalmatians (1996)  101 Dalmatians (One Hundred and One Dalmatians) (1961)  102 Dalmatians (2000)  12 Angry Men (1957)  \
userId                                                                                                                                                                                                                                                                                                     
108170.0                 NaN                          NaN                             NaN                            NaN                                NaN               NaN                    NaN                                                NaN                         NaN                  NaN   

title     12 Years a Slave (2013)  127 Hours (2010)  13 Going on 30 (2004)  13th Warrior, The (1999)  1408 (2007)  15 Minutes (2001)  16 Blocks (2006)  17 Again (2009)  1984 (Nineteen Eighty-Four) (1984)  2 Days in the Valley (1996)  2 Fast 2 Furious (Fast and the Furious 2, The) (2003)  \
userId                                                                                                                                                                                                                                                                                            
108170.0                      NaN               NaN                    NaN                       NaN          NaN                NaN               NaN              NaN                                 NaN                          NaN                                                NaN       

title     20,000 Leagues Under the Sea (1954)  200 Cigarettes (1999)  2001: A Space Odyssey (1968)  2010: The Year We Make Contact (1984)  2012 (2009)  2046 (2004)  21 (2008)  21 Grams (2003)  21 Jump Street (2012)  24 Hour Party People (2002)  25th Hour (2002)  27 Dresses (2008)  28 Days (2000)  \
userId                                                                                                                                                                                                                                                                                                     
108170.0                                  NaN                    NaN                           5.0                                    NaN          NaN          NaN        NaN              NaN                    NaN                          NaN               NaN                NaN             NaN   

title     28 Days Later (2002)  28 Weeks Later (2007)  3 Ninjas (1992)  3-Iron (Bin-jip) (2004)  30 Days of Night (2007)  300 (2007)  3000 Miles to Graceland (2001)  39 Steps, The (1935)  3:10 to Yuma (2007)  40 Days and 40 Nights (2002)  40-Year-Old Virgin, The (2005)  \
userId                                                                                                                                                                                                                                                                          
108170.0                   NaN                    NaN              NaN                      NaN                      NaN         NaN                             NaN                   NaN                  NaN                           NaN                             NaN   

title     400 Blows, The (Les quatre cents coups) (1959)  42 Up (1998)  48 Hrs. (1982)  50 First Dates (2004)  50/50 (2011)  54 (1998)  6th Day, The (2000)  7th Voyage of Sinbad, The (1958)  8 1/2 (8½) (1963)  8 Heads in a Duffel Bag (1997)  8 Mile (2002)  8MM (1999)  9 (2009)  \
userId                                                                                                                                                                                                                         

In [65]:
# random_user_df'in shapesine bakalım;
random_user_df.shape

(1, 3159)

In [66]:
# Seçilen kullanıcının oy kullandığı filmleri tespit edelim;
random_user_df.notnull().any()

title
'burbs, The (1989)                   False
(500) Days of Summer (2009)          False
*batteries not included (1987)       False
...And Justice for All (1979)        False
10 Things I Hate About You (1999)    False
                                     ...  
Zulu (1964)                          False
[REC] (2007)                         False
eXistenZ (1999)                      False
xXx (2002)                           False
¡Three Amigos! (1986)                False
Length: 3159, dtype: bool

In [67]:
# Bunları bir liste içerisine alalım;
random_user_df.columns[random_user_df.notnull().any()]

# "random_user_df.columns" bütün sütunları getirirken,
# içerisine girdiğimiz koşuldan true olanları bir liste içerine getiriyoruz.

Index(['2001: A Space Odyssey (1968)', 'Adventures of Priscilla, Queen of the Desert, The (1994)', 'Akira (1988)', 'Aladdin (1992)', 'Aliens (1986)', 'Almost Famous (2000)', 'Along Came Polly (2004)', 'Amadeus (1984)', 'American History X (1998)', 'Any Given Sunday (1999)',
       ...
       'Untouchables, The (1987)', 'Usual Suspects, The (1995)', 'Wallace & Gromit: A Close Shave (1995)', 'Wallace & Gromit: The Best of Aardman Animation (1996)', 'Wallace & Gromit: The Wrong Trousers (1993)', 'Wedding Singer, The (1998)', 'Who Framed Roger Rabbit? (1988)',
       'Wild at Heart (1990)', 'Willow (1988)', 'X2: X-Men United (2003)'],
      dtype='object', name='title', length=186)

In [68]:
# Yukarıdaki index'i normal bir listeye çevirip bir değişkene atayalım;
movies_watched = random_user_df.columns[random_user_df.notnull().any()].to_list()
movies_watched

['2001: A Space Odyssey (1968)',
 'Adventures of Priscilla, Queen of the Desert, The (1994)',
 'Akira (1988)',
 'Aladdin (1992)',
 'Aliens (1986)',
 'Almost Famous (2000)',
 'Along Came Polly (2004)',
 'Amadeus (1984)',
 'American History X (1998)',
 'Any Given Sunday (1999)',
 'Arachnophobia (1990)',
 'Army of Darkness (1993)',
 'Bad Santa (2003)',
 'Basic Instinct (1992)',
 'Batman Returns (1992)',
 'Big Lebowski, The (1998)',
 'Blair Witch Project, The (1999)',
 'Blues Brothers, The (1980)',
 'Boondock Saints, The (2000)',
 'Bourne Identity, The (2002)',
 'Braveheart (1995)',
 'Breakfast Club, The (1985)',
 "Carlito's Way (1993)",
 'Catwoman (2004)',
 "Charlie's Angels (2000)",
 'City Slickers (1991)',
 "City Slickers II: The Legend of Curly's Gold (1994)",
 'Clear and Present Danger (1994)',
 'Clerks (1994)',
 'Clockwork Orange, A (1971)',
 'Con Air (1997)',
 'Contact (1997)',
 'Copycat (1995)',
 'Crocodile Dundee (1986)',
 'Cruel Intentions (1999)',
 'Crying Game, The (1992)',
 'D

# Görev 3:

In [69]:
# Seçilen kullanıcının izlediği fimlere ait sutunları user_movie_df'ten seçelim, 
# ve movies_watched_df adında yeni bir dataframe oluşturalım;
movies_watched_df = user_movie_df[movies_watched]

# Verimize hızlı bir bakış atalım;
movies_watched_df.head()


title,2001: A Space Odyssey (1968),"Adventures of Priscilla, Queen of the Desert, The (1994)",Akira (1988),Aladdin (1992),Aliens (1986),Almost Famous (2000),Along Came Polly (2004),Amadeus (1984),American History X (1998),Any Given Sunday (1999),Arachnophobia (1990),Army of Darkness (1993),Bad Santa (2003),Basic Instinct (1992),Batman Returns (1992),"Big Lebowski, The (1998)","Blair Witch Project, The (1999)","Blues Brothers, The (1980)","Boondock Saints, The (2000)","Bourne Identity, The (2002)",Braveheart (1995),"Breakfast Club, The (1985)",Carlito's Way (1993),Catwoman (2004),Charlie's Angels (2000),City Slickers (1991),City Slickers II: The Legend of Curly's Gold (1994),Clear and Present Danger (1994),Clerks (1994),"Clockwork Orange, A (1971)",Con Air (1997),Contact (1997),Copycat (1995),Crocodile Dundee (1986),Cruel Intentions (1999),"Crying Game, The (1992)","Dark Crystal, The (1982)",Dawn of the Dead (2004),Dead Poets Society (1989),Deliverance (1972),Demolition Man (1993),Desperado (1995),"Devil's Advocate, The (1997)",Dirty Rotten Scoundrels (1988),Donnie Darko (2001),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Easy Rider (1969),Edward Scissorhands (1990),Eternal Sunshine of the Spotless Mind (2004),Evil Dead II (Dead by Dawn) (1987),Ferris Bueller's Day Off (1986),Fight Club (1999),Finding Nemo (2003),Finding Neverland (2004),First Knight (1995),"Fisher King, The (1991)",Flight of the Navigator (1986),Freddy Got Fingered (2001),"Frighteners, The (1996)",From Dusk Till Dawn (1996),Gangs of New York (2002),Gattaca (1997),Ghostbusters (a.k.a. Ghost Busters) (1984),Gladiator (2000),Goldfinger (1964),Goodfellas (1990),"Grand Day Out with Wallace and Gromit, A (1989)",Grease (1978),Grease 2 (1982),"Green Mile, The (1999)",Grosse Pointe Blank (1997),Harry Potter and the Prisoner of Azkaban (2004),Hero (Ying xiong) (2002),Highlander (1986),"Honey, I Shrunk the Kids (1989)","Hunt for Red October, The (1990)",Indiana Jones and the Last Crusade (1989),"Italian Job, The (2003)",Kill Bill: Vol. 1 (2003),Kill Bill: Vol. 2 (2004),Kindergarten Cop (1990),Kingpin (1996),"Knight's Tale, A (2001)",Labyrinth (1986),Last Action Hero (1993),"Last Samurai, The (2003)",Leaving Las Vegas (1995),Lemony Snicket's A Series of Unfortunate Events (2004),Liar Liar (1997),Life Is Beautiful (La Vita è bella) (1997),"Little Mermaid, The (1989)",Live and Let Die (1973),"Lord of the Rings: The Fellowship of the Ring, The (2001)","Lord of the Rings: The Return of the King, The (2003)","Lord of the Rings: The Two Towers, The (2002)",Léon: The Professional (a.k.a. The Professional) (Léon) (1994),Master and Commander: The Far Side of the World (2003),"Matrix, The (1999)",Meet the Parents (2000),Memento (2000),Mickey Blue Eyes (1999),Minority Report (2002),Misery (1990),Mission: Impossible (1996),Mission: Impossible II (2000),"Monsters, Inc. (2001)",Monty Python Live at the Hollywood Bowl (1982),Monty Python and the Holy Grail (1975),Monty Python's Life of Brian (1979),Mulholland Drive (2001),Muriel's Wedding (1994),My Cousin Vinny (1992),"Naked Gun: From the Files of Police Squad!, The (1988)",Nine Months (1995),Ocean's Eleven (2001),"Others, The (2001)",Pirates of the Caribbean: The Curse of the Black Pearl (2003),Police Academy 5: Assignment: Miami Beach (1988),Police Academy 6: City Under Siege (1989),"Princess Bride, The (1987)",Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Rain Man (1988),Reservoir Dogs (1992),Ringu (Ring) (1998),Rob Roy (1995),Robin Hood: Men in Tights (1993),Romancing the Stone (1984),Rumble in the Bronx (Hont faan kui) (1995),Saving Private Ryan (1998),Scarface (1983),Schindler's List (1993),"Secret of NIMH, The (1982)",Seven (a.k.a. Se7en) (1995),Shaun of the Dead (2004),"Shawshank Redemption, The (1994)","Shining, The (1980)",Shrek (2001),Shrek 2 (2004),"Silence of the Lambs, The (1991)","Sixth Sense, The (1999)",Snatch (2000),Sneakers (1992),So I Married an Axe Mur

In [70]:
# df'in shapesine bakalım;
movies_watched_df.shape

(138493, 186)

In [71]:
# Her bir kullancının seçili user'in izlediği filmlerin kaçını izlediği bilgisini taşıyan,
# user_movie_count adında yeni bir dataframe oluşturalım;
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count.head()

userId
1.0    53
2.0    11
3.0    47
4.0     5
5.0    16
dtype: int64

In [72]:
# indekse reset atalım ve sütun isimlerini değiştirelim;
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]

# Verimize hızlı bir bakış atalım;
user_movie_count.head()

,userId,movie_count
0,1.0,53
1,2.0,11
2,3.0,47
3,4.0,5
4,5.0,16


In [73]:
# user_movie_count'in shapesine bakalım;

user_movie_count.shape

(138493, 2)

In [74]:
# Seçilen kullanıcının oy verdiği filmlerin yüzde 60 ve üstünü izleyenleri benzer kullanıcılar olarak düşünelim.
# Bu kullanıcıların id’lerinden users_same_movies adında bir liste oluşturalım;
perc = len(movies_watched) * 60 / 100
users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]
users_same_movies

90            91.0
115          116.0
155          156.0
293          294.0
297          298.0
            ...   
138133    138134.0
138207    138208.0
138253    138254.0
138324    138325.0
138410    138411.0
Name: userId, Length: 2326, dtype: float64

    2326 kişi seçtiğimiz random_user'ın izlediği filmlerin %60'ını izlemiştir.

In [75]:
# user_same_movies listesi içerisindeki seçili user ile benzerlik gösteren kullanıcıların
# id’lerinin bulunacağı şekilde movies_watched_df dataframe’ini filtreleyelim;
final_df = movies_watched_df[movies_watched_df.index.isin(users_same_movies)]
final_df.head()

title,2001: A Space Odyssey (1968),"Adventures of Priscilla, Queen of the Desert, The (1994)",Akira (1988),Aladdin (1992),Aliens (1986),Almost Famous (2000),Along Came Polly (2004),Amadeus (1984),American History X (1998),Any Given Sunday (1999),Arachnophobia (1990),Army of Darkness (1993),Bad Santa (2003),Basic Instinct (1992),Batman Returns (1992),"Big Lebowski, The (1998)","Blair Witch Project, The (1999)","Blues Brothers, The (1980)","Boondock Saints, The (2000)","Bourne Identity, The (2002)",Braveheart (1995),"Breakfast Club, The (1985)",Carlito's Way (1993),Catwoman (2004),Charlie's Angels (2000),City Slickers (1991),City Slickers II: The Legend of Curly's Gold (1994),Clear and Present Danger (1994),Clerks (1994),"Clockwork Orange, A (1971)",Con Air (1997),Contact (1997),Copycat (1995),Crocodile Dundee (1986),Cruel Intentions (1999),"Crying Game, The (1992)","Dark Crystal, The (1982)",Dawn of the Dead (2004),Dead Poets Society (1989),Deliverance (1972),Demolition Man (1993),Desperado (1995),"Devil's Advocate, The (1997)",Dirty Rotten Scoundrels (1988),Donnie Darko (2001),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Easy Rider (1969),Edward Scissorhands (1990),Eternal Sunshine of the Spotless Mind (2004),Evil Dead II (Dead by Dawn) (1987),Ferris Bueller's Day Off (1986),Fight Club (1999),Finding Nemo (2003),Finding Neverland (2004),First Knight (1995),"Fisher King, The (1991)",Flight of the Navigator (1986),Freddy Got Fingered (2001),"Frighteners, The (1996)",From Dusk Till Dawn (1996),Gangs of New York (2002),Gattaca (1997),Ghostbusters (a.k.a. Ghost Busters) (1984),Gladiator (2000),Goldfinger (1964),Goodfellas (1990),"Grand Day Out with Wallace and Gromit, A (1989)",Grease (1978),Grease 2 (1982),"Green Mile, The (1999)",Grosse Pointe Blank (1997),Harry Potter and the Prisoner of Azkaban (2004),Hero (Ying xiong) (2002),Highlander (1986),"Honey, I Shrunk the Kids (1989)","Hunt for Red October, The (1990)",Indiana Jones and the Last Crusade (1989),"Italian Job, The (2003)",Kill Bill: Vol. 1 (2003),Kill Bill: Vol. 2 (2004),Kindergarten Cop (1990),Kingpin (1996),"Knight's Tale, A (2001)",Labyrinth (1986),Last Action Hero (1993),"Last Samurai, The (2003)",Leaving Las Vegas (1995),Lemony Snicket's A Series of Unfortunate Events (2004),Liar Liar (1997),Life Is Beautiful (La Vita è bella) (1997),"Little Mermaid, The (1989)",Live and Let Die (1973),"Lord of the Rings: The Fellowship of the Ring, The (2001)","Lord of the Rings: The Return of the King, The (2003)","Lord of the Rings: The Two Towers, The (2002)",Léon: The Professional (a.k.a. The Professional) (Léon) (1994),Master and Commander: The Far Side of the World (2003),"Matrix, The (1999)",Meet the Parents (2000),Memento (2000),Mickey Blue Eyes (1999),Minority Report (2002),Misery (1990),Mission: Impossible (1996),Mission: Impossible II (2000),"Monsters, Inc. (2001)",Monty Python Live at the Hollywood Bowl (1982),Monty Python and the Holy Grail (1975),Monty Python's Life of Brian (1979),Mulholland Drive (2001),Muriel's Wedding (1994),My Cousin Vinny (1992),"Naked Gun: From the Files of Police Squad!, The (1988)",Nine Months (1995),Ocean's Eleven (2001),"Others, The (2001)",Pirates of the Caribbean: The Curse of the Black Pearl (2003),Police Academy 5: Assignment: Miami Beach (1988),Police Academy 6: City Under Siege (1989),"Princess Bride, The (1987)",Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Rain Man (1988),Reservoir Dogs (1992),Ringu (Ring) (1998),Rob Roy (1995),Robin Hood: Men in Tights (1993),Romancing the Stone (1984),Rumble in the Bronx (Hont faan kui) (1995),Saving Private Ryan (1998),Scarface (1983),Schindler's List (1993),"Secret of NIMH, The (1982)",Seven (a.k.a. Se7en) (1995),Shaun of the Dead (2004),"Shawshank Redemption, The (1994)","Shining, The (1980)",Shrek (2001),Shrek 2 (2004),"Silence of the Lambs, The (1991)","Sixth Sense, The (1999)",Snatch (2000),Sneakers (1992),So I Married an Axe Mur

In [76]:
# final_df'in shapesine bakalım;
final_df.shape

(2326, 186)

In [77]:
# Kullanıcıların birbirleri ile olan korelasyonlarının bulunacağı yeni bir corr_df dataframe’i oluşturalım;
final_df.T.corr()

# Kullanıcıların korelasyonu oluşturulacağı için trasnpose aldık.

userId    91.0      116.0     156.0     294.0     298.0     359.0     367.0     388.0     586.0     614.0     648.0     741.0     768.0     775.0     812.0     903.0     971.0     982.0     1185.0    1200.0    1411.0    1419.0    1421.0    1507.0    1516.0    1715.0    1748.0    1763.0    \
userId                                                                                                                                                                                                                                                                                             
91.0      1.000000  0.454166  0.322393  0.338482  0.152716  0.245899  0.505629  0.438347  0.188644  0.572365  0.130954  0.458746  0.435516  0.248756  0.210939  0.427807  0.270179  0.346831  0.409081  0.357387  0.285924  0.406923  0.349751  0.348747  0.476192  0.227252  0.311110  0.141599   
116.0     0.454166  1.000000  0.460489  0.173071  0.347831  0.186010  0.539608  0.639885  0.594176  0.445155  0.237793  0.470893  0.521484  0.324930  0.219015  0.487399  0.428444  0.529501  0.518550  0.523892  0.475841  0.392963  0.533668  0.529703  0.271898  0.285976  0.201087  0.145622   
156.0     0.322393  0.460489  1.000000  0.219779  0.339833  0.201932  0.499251  0.416804  0.369159  0.317214  0.198428  0.424603  0.455694  0.370905  0.125030  0.377045  0.207375  0.416951  0.415380  0.359369  0.236454  0.341773  0.272729  0.531793  0.451023  0.316476  0.204207  0.252350   
294.0     0.338482  0.173071  0.219779  1.000000  0.276074 -0.046278  0.422012  0.239812  0.224724  0.151781  0.180566  0.265016  0.376644  0.239418  0.250364  0.172083  0.112978  0.194375  0.251347  0.228026  0.151239  0.081787  0.141986  0.373929  0.151566  0.173497  0.117709  0.031635   
298.0     0.152716  0.347831  0.339833  0.276074  1.000000  0.029420  0.400850  0.170610  0.330291 -0.050431  0.087147  0.198776  0.291661  0.201714  0.155462  0.253232  0.237256  0.318140  0.355499  0.157646  0.080409  0.201560  0.002581  0.253144  0.165723  0.074994  0.315457  0.083437   
...            ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...       ...   
138134.0  0.267141  0.419395  0.041885  0.137793  0.043679  0.176268  0.146220  0.302073  0.229548  0.346178  0.269861  0.244567  0.303591  0.432589 -0.056377  0.465036  0.336329  0.331916  0.262554  0.223817  0.274606  0.086867  0.405416  0.263747  0.077668  0.237070  0.127965  0.145715   
138208.0  0.548877  0.420043  0.346490  0.212778  0.215007  0.200468  0.472367  0.541147  0.323386  0.409151  0.219103  0.443416  0.322211  0.390484  0.284974  0.461101  0.333395  0.474708  0.424160  0.422319  0.517930  0.426533  0.389468  0.389924  0.276914  0.371639  0.221336  0.256650   
138254.0  0.199193  0.568327  0.439030  0.286854  0.333704 -0.070669  0.555171  0.599660  0.509531  0.120046  0.171260  0.334821  0.543012  0.222198  0.068203  0.347057  0.408481  0.485267  0.341874  0.462308  0.329198  0.348356  0.366699  0.461892  0.161344  0.162406  0.322448  0.308628   
138325.0  0.371530  0.481510  0.342354  0.322522  0.036160  0.503810  0.406140  0.367201  0.406796  0.237543  0.407043  0.516453  0.429399  0.366555  0.178500  0.499837  0.329397  0.521236  0.538107  0.454747  0.374859  0.268557  0.333478  0.414471  0.164909  0.448013  0.211376  0.168724   
138411.0  0.521917  0.477922  0.441789  0.307772  0.160045  0.408717  0.367519  0.455521  0.361254  0.282996  0.272729  0.433618  0.363674  0.484408  0.150117  0.451579  0.378964  0.373713  0.343643  0.352352  0.470427  0.355608  0.464270  0.443431  0.415245  0.367104  0.386742  0.284311   

userId    1849.0    1906.0    1966.0    1972.0    2038.0    2071.0    2099.0    2158.0    2232.0    2261.0    2344.0    2397.0    2657.0    2669.0    2772.0    2945.0    2988.0    3171.0    3218.0    328

In [78]:
# Transpose almazsak, filmlerin korelasyonunu bulur;

final_df.corr().head(2)

title,2001: A Space Odyssey (1968),"Adventures of Priscilla, Queen of the Desert, The (1994)",Akira (1988),Aladdin (1992),Aliens (1986),Almost Famous (2000),Along Came Polly (2004),Amadeus (1984),American History X (1998),Any Given Sunday (1999),Arachnophobia (1990),Army of Darkness (1993),Bad Santa (2003),Basic Instinct (1992),Batman Returns (1992),"Big Lebowski, The (1998)","Blair Witch Project, The (1999)","Blues Brothers, The (1980)","Boondock Saints, The (2000)","Bourne Identity, The (2002)",Braveheart (1995),"Breakfast Club, The (1985)",Carlito's Way (1993),Catwoman (2004),Charlie's Angels (2000),City Slickers (1991),City Slickers II: The Legend of Curly's Gold (1994),Clear and Present Danger (1994),Clerks (1994),"Clockwork Orange, A (1971)",Con Air (1997),Contact (1997),Copycat (1995),Crocodile Dundee (1986),Cruel Intentions (1999),"Crying Game, The (1992)","Dark Crystal, The (1982)",Dawn of the Dead (2004),Dead Poets Society (1989),Deliverance (1972),Demolition Man (1993),Desperado (1995),"Devil's Advocate, The (1997)",Dirty Rotten Scoundrels (1988),Donnie Darko (2001),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Easy Rider (1969),Edward Scissorhands (1990),Eternal Sunshine of the Spotless Mind (2004),Evil Dead II (Dead by Dawn) (1987),Ferris Bueller's Day Off (1986),Fight Club (1999),Finding Nemo (2003),Finding Neverland (2004),First Knight (1995),"Fisher King, The (1991)",Flight of the Navigator (1986),Freddy Got Fingered (2001),"Frighteners, The (1996)",From Dusk Till Dawn (1996),Gangs of New York (2002),Gattaca (1997),Ghostbusters (a.k.a. Ghost Busters) (1984),Gladiator (2000),Goldfinger (1964),Goodfellas (1990),"Grand Day Out with Wallace and Gromit, A (1989)",Grease (1978),Grease 2 (1982),"Green Mile, The (1999)",Grosse Pointe Blank (1997),Harry Potter and the Prisoner of Azkaban (2004),Hero (Ying xiong) (2002),Highlander (1986),"Honey, I Shrunk the Kids (1989)","Hunt for Red October, The (1990)",Indiana Jones and the Last Crusade (1989),"Italian Job, The (2003)",Kill Bill: Vol. 1 (2003),Kill Bill: Vol. 2 (2004),Kindergarten Cop (1990),Kingpin (1996),"Knight's Tale, A (2001)",Labyrinth (1986),Last Action Hero (1993),"Last Samurai, The (2003)",Leaving Las Vegas (1995),Lemony Snicket's A Series of Unfortunate Events (2004),Liar Liar (1997),Life Is Beautiful (La Vita è bella) (1997),"Little Mermaid, The (1989)",Live and Let Die (1973),"Lord of the Rings: The Fellowship of the Ring, The (2001)","Lord of the Rings: The Return of the King, The (2003)","Lord of the Rings: The Two Towers, The (2002)",Léon: The Professional (a.k.a. The Professional) (Léon) (1994),Master and Commander: The Far Side of the World (2003),"Matrix, The (1999)",Meet the Parents (2000),Memento (2000),Mickey Blue Eyes (1999),Minority Report (2002),Misery (1990),Mission: Impossible (1996),Mission: Impossible II (2000),"Monsters, Inc. (2001)",Monty Python Live at the Hollywood Bowl (1982),Monty Python and the Holy Grail (1975),Monty Python's Life of Brian (1979),Mulholland Drive (2001),Muriel's Wedding (1994),My Cousin Vinny (1992),"Naked Gun: From the Files of Police Squad!, The (1988)",Nine Months (1995),Ocean's Eleven (2001),"Others, The (2001)",Pirates of the Caribbean: The Curse of the Black Pearl (2003),Police Academy 5: Assignment: Miami Beach (1988),Police Academy 6: City Under Siege (1989),"Princess Bride, The (1987)",Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Rain Man (1988),Reservoir Dogs (1992),Ringu (Ring) (1998),Rob Roy (1995),Robin Hood: Men in Tights (1993),Romancing the Stone (1984),Rumble in the Bronx (Hont faan kui) (1995),Saving Private Ryan (1998),Scarface (1983),Schindler's List (1993),"Secret of NIMH, The (1982)",Seven (a.k.a. Se7en) (1995),Shaun of the Dead (2004),"Shawshank Redemption, The (1994)","Shining, The (1980)",Shrek (2001),Shrek 2 (2004),"Silence of the Lambs, The (1991)","Sixth Sense, The (1999)",Snatch (2000),Sneakers (1992),So I Married an Axe Mur

In [79]:
# unstack() ile pivot table'ı hiyerarşik düzene dönüştürelim;

final_df.T.corr().unstack()

userId    userId  
91.0      91.0        1.000000
          116.0       0.454166
          156.0       0.322393
          294.0       0.338482
          298.0       0.152716
                        ...   
138411.0  138134.0    0.361395
          138208.0    0.449004
          138254.0    0.308810
          138325.0    0.473589
          138411.0    1.000000
Length: 5410276, dtype: float64

In [80]:
# Bu tablodaki değerleri sıralayalım;
corr_df = final_df.T.corr().unstack().sort_values()
corr_df

userId    userId  
110130.0  72838.0    -0.581570
72838.0   110130.0   -0.581570
33581.0   100618.0   -0.481252
100618.0  33581.0    -0.481252
126121.0  89242.0    -0.476220
                        ...   
91893.0   91893.0     1.000000
91867.0   91867.0     1.000000
91793.0   91793.0     1.000000
90381.0   90381.0     1.000000
138411.0  138411.0    1.000000
Length: 5410276, dtype: float64

In [81]:
# Bu pandas serisini df'ye çevirelim;
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df

,,corr
userId,userId,
110130.0,72838.0,-0.581570
72838.0,110130.0,-0.581570
33581.0,100618.0,-0.481252
100618.0,33581.0,-0.481252
126121.0,89242.0,-0.476220
...,...,...
91893.0,91893.0,1.000000
91867.0,91867.0,1.000000
91793.0,91793.0,1.000000


In [82]:
# indeksleri isimlendirelim;
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df

,,corr
user_id_1,user_id_2,
110130.0,72838.0,-0.581570
72838.0,110130.0,-0.581570
33581.0,100618.0,-0.481252
100618.0,33581.0,-0.481252
126121.0,89242.0,-0.476220
...,...,...
91893.0,91893.0,1.000000
91867.0,91867.0,1.000000
91793.0,91793.0,1.000000


In [83]:
# indekse reset atalım;
corr_df = corr_df.reset_index()
corr_df

,user_id_1,user_id_2,corr
0,110130.0,72838.0,-0.581570
1,72838.0,110130.0,-0.581570
2,33581.0,100618.0,-0.481252
3,100618.0,33581.0,-0.481252
4,126121.0,89242.0,-0.476220
...,...,...,...
5410271,91893.0,91893.0,1.000000
5410272,91867.0,91867.0,1.000000
5410273,91793.0,91793.0,1.000000
5410274,90381.0,90381.0,1.000000


In [84]:
# Seçili kullanıcı ile yüksek korelasyona sahip (0.65’in üzerinde olan) kullanıcıları filtreleyerek, 
# top_users adında yeni bir dataframe oluşturalım;
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][["user_id_2", "corr"]].  \
    reset_index(drop=True)
top_users

,user_id_2,corr
0,82860.0,0.654646
1,12710.0,0.663332
2,121747.0,0.673295
3,119285.0,0.700081
4,11517.0,0.707652
5,5155.0,0.716406
6,108170.0,1.000000


In [85]:
# Bu korelasyon katsayılarını büyükten küçüğe sıralayalım;
top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
top_users.head()

,userId,corr
6,108170.0,1.000000
5,5155.0,0.716406
4,11517.0,0.707652
3,119285.0,0.700081
2,121747.0,0.673295


In [86]:
# top_users shape'sine bakalım;
top_users.shape

(7, 2)

In [87]:
# top_users dataframe’ine rating veri seti ile merge edelim;
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
top_users_ratings

,userId,corr,movieId,rating
0,108170.0,1.000000,1,3.0
1,108170.0,1.000000,22,3.0
2,108170.0,1.000000,25,5.0
3,108170.0,1.000000,32,5.0
4,108170.0,1.000000,47,5.0
...,...,...,...,...
4752,82860.0,0.654646,8905,2.0
4753,82860.0,0.654646,8906,2.5
4754,82860.0,0.654646,8947,3.0
4755,82860.0,0.654646,27317,3.0


In [91]:
# Kişinin kendi ile korelasyonu 1 çıkar, bu sebeple kendisini bu df'den çıkaralım;
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]
top_users_ratings.sort_values(by=["corr", "rating"], ascending=False)

,userId,corr,movieId,rating
242,5155.0,0.716406,180,5.0
250,5155.0,0.716406,223,5.0
251,5155.0,0.716406,231,5.0
261,5155.0,0.716406,288,5.0
263,5155.0,0.716406,296,5.0
...,...,...,...,...
4699,82860.0,0.654646,6793,0.5
4700,82860.0,0.654646,6794,0.5
4713,82860.0,0.654646,7004,0.5
4739,82860.0,0.654646,7701,0.5


    Bu df'ye göre userId'si belli olan kişilerin yüksek puan verdiği filmleri random_user beğenebilir, düşük puan verdiği filmleri ise beğenmeyebilir.

### Görev 5:

In [92]:
# Her bir kullanıcının corr ve rating değerlerinin çarpımından oluşan 
# weighted_rating adında yeni bir değişken oluşturalım;

top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.head()

,userId,corr,movieId,rating,weighted_rating
194,5155.0,0.716406,1,3.5,2.507423
195,5155.0,0.716406,2,3.0,2.149219
196,5155.0,0.716406,5,3.0,2.149219
197,5155.0,0.716406,9,3.0,2.149219
198,5155.0,0.716406,10,4.0,2.865626


In [93]:
# Film id’sine her bir filme ait tüm kullanıcıların weighted rating’lerinin ortalama değerini içeren 
# recommendation_df adında yeni bir dataframe oluşturalım;
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

# index'e reset atalım;
recommendation_df = recommendation_df.reset_index()

# Verimize hızlı bir bakış atalım;
recommendation_df.head()

,movieId,weighted_rating
0,1,2.124196
1,2,1.319527
2,3,0.986312
3,5,1.484503
4,6,2.526375


In [95]:
# recommendation_df içerisinde weighted rating'i 3.5'ten büyük olan filmleri seçelim,
# ve weighted rating’e göre sıralayalım;
recommendation_df[recommendation_df["weighted_rating"] > 3.5]

# İlk 5 gözlemi movies_to_be_recommend olarak kaydediniz.
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5]. \
    sort_values("weighted_rating", ascending=False)

# Verimize bakalım;
movies_to_be_recommend

,movieId,weighted_rating
229,671,3.582032
924,3098,3.582032
1349,5110,3.582032
856,2804,3.582032
1785,26840,3.538261
2183,93838,3.538261
2026,60684,3.538261
1971,53000,3.538261
1961,51559,3.538261
1838,32196,3.538261


In [96]:
# Tavsiye edilen 5 filmin isimlerini getirelim;
movies_to_be_recommend.merge(movie[["movieId", "title"]])["title"][:5]

0    Mystery Science Theater 3000: The Movie (1996)
1                               Natural, The (1984)
2                             Super Troopers (2001)
3                         Christmas Story, A (1983)
4                       Sonatine (Sonachine) (1993)
Name: title, dtype: object

### Item-Based Recommendation

    Kullanıcının en son izlediği ve en yüksek puan verdiği filmin adına göre item-based öneri yapacağız.

In [97]:
# Kullanıcımızı tanımlayalım;
user = 108170

# df'leri baştan tanımlayalım;
movie = pd.read_csv('datasets/movie.csv')
rating = pd.read_csv('datasets/rating.csv')

In [98]:
# Öneri yapılacak kullanıcının 5 puan verdiği filmlerden puanı en güncel olan filmin id'sinin bulalım;
movie_id = rating[(rating["userId"] == user) & (rating["rating"] == 5.0)].  \
    sort_values(by="timestamp", ascending=False)["movieId"][0:1].values[0]

movie_id

7044

In [99]:
# User based recommendation bölümünde oluşturulan user_movie_df dataframe’ini seçilen film id’sine göre filtereleyelim;
movie[movie["movieId"] == movie_id]["title"].values[0]
movie_df = user_movie_df[movie[movie["movieId"] == movie_id]["title"].values[0]]
movie_df

userId
1.0        NaN
2.0        NaN
3.0        NaN
4.0        NaN
5.0        NaN
            ..
138489.0   NaN
138490.0   NaN
138491.0   NaN
138492.0   NaN
138493.0   NaN
Name: Wild at Heart (1990), Length: 138493, dtype: float64

In [100]:
# Filtrelenen dataframe’i kullanarak seçili filmle diğer filmlerin korelasyonunu bulunuz ve sıralayınız.
user_movie_df.corrwith(movie_df).sort_values(ascending=False).head(10)

title
Wild at Heart (1990)                     1.000000
My Science Project (1985)                0.570187
Mediterraneo (1991)                      0.538868
Old Man and the Sea, The (1958)          0.536192
National Lampoon's Senior Trip (1995)    0.533029
Clockwatchers (1997)                     0.483337
Repo Man (1984)                          0.478674
Lost Highway (1997)                      0.476251
Blue Velvet (1986)                       0.471225
Jeffrey (1995)                           0.457849
dtype: float64

In [101]:
# Son iki adımı fonksiyonlaştıralım;
def item_based_recommender(movie_name, user_movie_df):
    movie = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie).sort_values(ascending=False).head(10)

In [102]:
# Seçili film’in kendisi haricinde ilk 5 film’I öneri olarak verelim
movies_from_item_based = item_based_recommender(movie[movie["movieId"] == movie_id]["title"].values[0], user_movie_df)
movies_from_item_based

title
Wild at Heart (1990)                     1.000000
My Science Project (1985)                0.570187
Mediterraneo (1991)                      0.538868
Old Man and the Sea, The (1958)          0.536192
National Lampoon's Senior Trip (1995)    0.533029
Clockwatchers (1997)                     0.483337
Repo Man (1984)                          0.478674
Lost Highway (1997)                      0.476251
Blue Velvet (1986)                       0.471225
Jeffrey (1995)                           0.457849
dtype: float64

In [104]:
# 1'den 6'ya kadar. 0'da filmin kendisi var. Onu dışarda bıraktık.
movies_from_item_based[1:6]

title
My Science Project (1985)                0.570187
Mediterraneo (1991)                      0.538868
Old Man and the Sea, The (1958)          0.536192
National Lampoon's Senior Trip (1995)    0.533029
Clockwatchers (1997)                     0.483337
dtype: float64